In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !git clone "https://github.com/YushRajcoomar/WaveNet-torch.git"
# %cd WaveNet-torch/

In [3]:
# # ### Acquire training data
# !wget -O file1.zip "https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.001"
# !wget -O file2.zip "https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.002"
# !wget -O file3.zip "https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.003"
# !wget "https://mirg.city.ac.uk/datasets/magnatagatune/annotations_final.csv"

# !mkdir audio
# !cat file1.zip file2.zip file3.zip > audio/mp3_all.zip
# !unzip audio/mp3_all.zip
# !mv "annotations_final.csv" "audio/annotations_final.csv"

In [4]:
from wavenet_blocks import *
from wavenet_model import *
from wavenet_train import *
from dataloader import CustomAudioDataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchaudio import transforms as T
from torch.optim import Adam

In [5]:
import pandas as pd

df = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\wavenet\audio\annotations_final.csv',delimiter='\t')

In [6]:
# Path to your CSV file containing 'mp3_path' column
csv_path = r'C:\Users\yush\OneDrive\Desktop\papers\wavenet\audio\annotations_final.csv'
audio_df = pd.read_csv(csv_path,delimiter='\t')

# Example of waveform transformation (you can customize this)
input_channels = 128

transform = [
    T.MelSpectrogram(n_mels = input_channels)  # Convert waveform to Mel spectrogram
]

# Create an instance of your custom dataset
audio_dataset = CustomAudioDataset(audio_df, transform=transform)

trainNumbers=int(len(audio_dataset)*0.9)
trainDataset,testDataset=torch.utils.data.random_split(audio_dataset,[trainNumbers,len(audio_dataset)-trainNumbers])
trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=8,shuffle=True)
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=8,shuffle=True)

c:\Users\yush\OneDrive\Desktop\papers\wavenet\wavenet\lib\site-packages\torchaudio\functional\functional.py:572: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


In [7]:
model = WaveNetModel(in_channels=input_channels,out_channels=1,skip_channels=10,kernel_size=3,residual_blocks=5)
optimizer = Adam(model.parameters(),lr=0.0001)
loss_func = torch.nn.MSELoss()
trainer = WaveNetTrain(model,optimizer,loss_func)

To-Do:
- Figure out this dimension issue. 
- Overfit single batch

In [10]:
for batch in trainDataLoader:
  x,y = batch
  trainer.fit(x,y)
  break

c:\Users\yush\OneDrive\Desktop\papers\wavenet\wavenet\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([8, 128, 241])) that is different to the input size (torch.Size([8, 1, 2090])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2090) must match the size of tensor b (241) at non-singleton dimension 2

In [24]:
str(torchaudio.get_audio_backend())

'None'

In [ ]:
from sklearn.model_selection import train_test_split


# Split the dataset into train and test sets
train_dataset, test_dataset = train_test_split(audio_dataset, test_size=0.2, random_state=42)

# Create train and test dataloaders
batch_size = 4  # Set your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

KeyboardInterrupt: 